In [22]:
import pandas as pd
import numpy as np

In [56]:
df = pd.read_excel("../data/csv/sampled_wells.xlsx", converters={'Collection Date': str,
                                                        'City': lambda x: str(x.strip()),
                                                        'ZipCode': str})

In [57]:
# number of records

len(df)

1714

In [58]:
# select columns

df = df[["Address",
         "City",
         "ZipCode",
         "Well Permit #",
         "Collection Date",
         "Arsenic",
         "pH"]]

In [59]:
# rename columns

df.columns = ["add","city","zip","id","date","ar","ph"]

In [60]:
# Check for missing values

df.isnull().sum()

add       0
city      1
zip       0
id      614
date      0
ar       13
ph       11
dtype: int64

In [61]:
# check for typos in city names

set(df['city'])

{'ALEXIS',
 'BELMONT',
 'BESSEMER CITY',
 'Bassemer City',
 'Belmont',
 'Bessemer City',
 'CHERRYVILLE',
 'CRAMERTON',
 'CROUSE',
 'Cherryville',
 'Crouse',
 'DALLAS',
 'Dallas',
 'GASTONIA',
 'Gastonia',
 'IRON STATION',
 'KINGS MOUNTAIN',
 'KINGS MOUTAIN',
 'King Mtn 1Kings Mountain',
 'Kings Mountain',
 'LINCOLNTON',
 'LOWEL',
 'LOWELL',
 'Lincolnton',
 'MC ADENVILLE',
 'MOUNT HOLLY',
 'MT HOLLY',
 'Mount Holly',
 'Mt. Holly',
 'PINEVILLE',
 'STALEY',
 'STANLEY',
 'Stanley',
 nan}

In [62]:
# correct typos for city

li = []

for each in df['city']:

    if each in ['GASTONIA', 'Gastonia']:
        li.append('GASTONIA')

    elif each in ['BELMONT', 'Belmont']:
        li.append('BELMONT')

    elif each in ['DALLAS', 'Dallas']:
        li.append('DALLAS')
    
    elif each in ['MOUNT HOLLY', 'Mt. Holly','MT HOLLY', 'Mount Holly']:
        li.append('MT HOLLY')

    elif each in ['STANLEY','Stanley', 'STALEY']:
        li.append('STANLEY')
    
    elif each in ['BESSEMER CITY', 'Bessemer City','Bassemer City']:
        li.append('BESSEMER CITY')
    
    elif each in ['KINGS MOUNTAIN', 'Kings Mountain','KINGS MOUTAIN','King Mtn 1Kings Mountain']:
        li.append('KINGS MTN')
    
    elif each in['CHERRYVILLE', 'Cherryville']:
        li.append('CHERRYVILLE')
    
    elif each in['LINCOLNTON','Lincolnton']:
        li.append('LINCOLNTON')
    
    elif each in['IRON STATION']:
        li.append('IRON STATION')

    elif each in['ALEXIS']:
        li.append('ALEXIS')

    elif each in['LOWEL','LOWELL']:
        li.append('LOWELL')

    elif each in['CROUSE', 'Crouse']:
        li.append('CROUSE')

    elif each in['CRAMERTON']: 
        li.append('CRAMERTON')

    elif each in ['MC ADENVILLE']:
        li.append('MCADENVILLE')

    elif each == 'PINEVILLE':
        li.append('PINEVILLE')
    else:
        li.append("")

df['city'] = li

df['city'].value_counts()

city
GASTONIA         360
BELMONT          285
DALLAS           254
MT HOLLY         183
STANLEY          168
BESSEMER CITY    162
KINGS MTN         99
CHERRYVILLE       90
LINCOLNTON        64
CROUSE            23
ALEXIS            16
LOWELL             3
CRAMERTON          3
PINEVILLE          1
MCADENVILLE        1
                   1
IRON STATION       1
Name: count, dtype: int64

In [64]:
# remove leading and trailing spaces from city names

df['city'] = [s.strip() for s in df['city']]

# drop rows with missing city values

df = df[df['city'] != '']

In [65]:
# save the cleaned data to a new file in data folder

df['sample_id'] = df.index

len(df)

1713

In [71]:
# check zip codes for typos (aka anything starting with 29 or not 5 digits)
df['zip'].value_counts()

zip
28012    277
28034    258
28056    198
28120    181
28164    165
28016    163
28052    142
28086     96
28021     91
28092     62
28033     23
28054     21
28006     15
28032      4
28098      3
28066      2
28101      1
29164      1
28210      1
28080      1
28093      1
2056       1
27355      1
28134      1
28102      1
28106      1
28806      1
28065      1
Name: count, dtype: int64

In [68]:
# convert zip column to string

df['zip'] = df['zip'].astype(str)


In [74]:
# check city and zip for bad values

valid_zips  = ['28006',
                '28012',
                '28016',
                '28021',
                '28032',
                '28033',
                '28034',
                '28052',
                '28053',
                '28054',
                '28055',
                '28056',
                '28077',
                '28086',
                '28092',
                '28098',
                '28101',
                '28120',
                '28164']


# loop through dataframe and if zip is not in valid_zips list, remove row



for index, row in df.iterrows():
    if row['zip'] not in valid_zips:
        df.drop(index, inplace=True)


# check number removed

len(df)

1700

In [89]:
# check for blanks in address

print(len(df[df['add'] == '']))
print(len(df[df['add'] == ' ']))

0
0


In [90]:
# trim leading and trailing spaces from address

df['add'] = [s.strip() for s in df['add']]

In [91]:
# Convert long date to short date

df['date'] = df['date'].str[:10]
df['date'] = [x.strip() for x in df['date']]

df['date'] = df['date'].replace(regex=['2/1/2021'], value="2021-02-01")

In [92]:
df['date']

0       2011-01-03
1       2011-01-04
3       2011-01-04
5       2011-01-24
6       2011-01-26
           ...    
1709    2021-04-05
1710    2021-02-08
1711    2021-02-08
1712    2021-02-15
1713    2020-07-29
Name: date, Length: 1687, dtype: object

In [93]:
# convert date to datetime

df['date_tested'] = pd.to_datetime(df['date'])

In [94]:
# create year column

df['year_tested'] = df['date_tested'].dt.year

In [95]:
# convert non-detect arsenic values to 0

li = []
for each in df['ar']:
    if each in('<0.001','< 0.005', '<0.005', '<0.01'):
        li.append(float(0))
    else:
        li.append(float(each))

df['ar'] = li

In [96]:
# Create a new column to group arsenic values into 0 and 1, 0 for <0.001 and 1 for >=0.001

li = []
for each in df['ar']:
    if each < 0.001:
        li.append('0')
    else: li.append('1')

df['group'] = li

df['group'].value_counts()

group
0    1569
1     118
Name: count, dtype: int64

In [97]:

# Create a new column to group arsenic values into 0 and 1, 0 for <0.005 and 1 for >=0.005

li = []
for each in df['ar']:
    if each < 0.005:
        li.append('0')
    else: li.append('1')

df['group_five'] = li


df['group_five'].value_counts()

group_five
0    1629
1      58
Name: count, dtype: int64

In [98]:

# Create a new column to group arsenic values into 0 and 1, 0 for <0.005 and 1 for >=0.01  (MCL)

li = []
for each in df['ar']:
    if each < 0.01:
        li.append('0')
    else: li.append('1')

df['group_mcl'] = li


df['group_mcl'].value_counts()

group_mcl
0    1656
1      31
Name: count, dtype: int64

In [99]:
len(df)

1687

In [100]:
# if missing arsenic or ph values, drop the row

df = df.dropna(subset=['ar','ph'])
len(df)

1687

In [101]:

df.to_csv("../data/alt/csv/sampled_wells_cleaned.csv", index=False)